In [1]:
#from elasticsearch_dsl import Search, Q, Range, connections
#from elasticsearch.client import Elasticsearch
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
from openai import OpenAI
import numpy as np

In [2]:
DATA_FOLDER = "../data/events/browse_ai/" 

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 50)
pd.set_option('display.max_colwidth', None)

In [4]:
df = pd.read_pickle(f"{DATA_FOLDER}df_browse_ai 2025-01-02_18-50-53.pkl")

In [5]:
extracted_columns = []
for col in df.columns:
    if col.startswith("extr"):
        extracted_columns.append(col)
    
extracted_columns.append("isOnMultipleDays")
extracted_columns

['extrStartDate',
 'extrStartTime',
 'extrEndTime',
 'extrEndDate',
 'extrMultipleDaysDetails',
 'extrCity',
 'extrStreetHouseNumber',
 'extrLocationDetails',
 'extrLocationName',
 'isOnMultipleDays']

In [6]:
id = 18075
df.loc[id, "isOnMultipleDays"]=False

In [7]:
id = 18072
df.loc[id, "isOnMultipleDays"]=False
df.loc[id, "extrStartTime"]="11"
df.loc[id, "extrEndTime"]="13"
df.loc[id, "extrMultipleDaysDetails"]=None

In [8]:
id = 4212
df.loc[id, "isOnMultipleDays"]=False
df.loc[id, "extrMultipleDaysDetails"]=None

In [9]:
id = 2289
df.loc[id, "extrStartDate"] = "2024-09-13"
df.loc[id, "extrEndDate"] = "2024-10-06"

In [10]:
# drop rows with undefined location
flt = df.extrLocationName=="verschiedene Veranstaltungsorte"
df = df[~flt]

In [11]:
cols_to_remove = []
for col in df.columns:
    if col.startswith("OpenAI") or col.startswith("count_") :
        cols_to_remove.append(col)
    
df = df.drop(columns=cols_to_remove, errors="ignore")

In [12]:
flt = (df.isOnMultipleDays==True) 
df_multiple_days = df[flt].copy()

In [13]:
# suppose missng end date in 2 weeks after start 
flt = (df.isOnMultipleDays==True) & (df.extrEndDate.isna() ) 
values = df.loc[flt, "extrStartDate"]
values = (pd.to_datetime(values) + pd.DateOffset(weeks=2)).dt.strftime("%Y-%m-%d")
values
df.loc[flt, "extrEndDate"] = values


In [14]:
# if end date is after 2024-10-31, set it to 2024-10-31
flt = (df.isOnMultipleDays==True) & (pd.to_datetime(df.extrEndDate) > pd.to_datetime("2024-10-31") )
df.loc[flt, "extrEndDate"]="2024-10-31" 

In [15]:
id = 4931
df.loc[id, "extrStartTime"]="20"
df.loc[id, "extrEndTime"]="23"

In [16]:
id = 6155
df.loc[id, "extrEndTime"]="22,5"

In [17]:
id = 3156
df.loc[id, "extrStartTime"]="15"

In [18]:
id= 18582
df.loc[id, "extrStartTime"]="15"
df.loc[id, "extrEndTime"]="16,5"
id=22687
df.loc[id, "extrStartTime"]="15"
df.loc[id, "extrEndTime"]="16,5"


In [19]:
id = 24499
df.loc[id, "extrStartTime"]="13"
df.loc[id, "extrEndTime"]="20,5"

In [20]:
id = 10469
df.loc[id, "extrLocationName"]="Münsterplatz"

In [21]:
id = 18075
df.loc[id, "extrMultipleDaysDetails"]=None

In [22]:
df.extrStartTime.unique()

array(['15:00', '10', '11', '8', '9', '12', '20', '17', '15', '16',
       '19,5', '10,5', '19', '18', nan, '11:45', '13:00', '14', '13',
       '12,5', '11,5', '22', '17:00', '15,5', '11:00', '20:00',
       '17:00 (for the opening ceremony)', '13,5', '19:00', '10:00',
       '18:00', '16,5', '18,5', '17,5', '21',
       'Termin/Uhrzeit\nSa. 21. Sep 2024\n21:30 Uhr',
       'Termin/Uhrzeit\nSo. 13. Okt 2024\n17:00 Uhr',
       'Termin/Uhrzeit\nSa. 12. Okt 2024\n20:00 Uhr',
       'Termin/Uhrzeit\nMi. 16. Okt 2024\n20:00 Uhr',
       'Termin/Uhrzeit\nSa. 19. Okt 2024\n20:00 Uhr',
       'Termin/Uhrzeit\nDo. 26. Sep 2024\n20:00 Uhr',
       'Termin/Uhrzeit\nSo. 27. Okt 2024\n19:00 Uhr', '16:00'],
      dtype=object)

In [23]:
flt = df.extrStartTime.str.endswith(":00", na=False)
values = df.loc[flt, "extrStartTime"].str.replace(":00", "")
values
df.loc[flt, "extrStartTime"] = values

In [24]:
flt = df.extrEndTime.str.endswith(":00", na=False)
values = df.loc[flt, "extrEndTime"].str.replace(":00", "")
values
df.loc[flt, "extrEndTime"] = values

In [25]:
flt = df.extrEndTime.str.endswith(":30", na=False)
values = df.loc[flt, "extrEndTime"].str.replace(":30", ",5")
values
df.loc[flt, "extrEndTime"] = values

In [26]:
flt = df.extrEndTime.str.endswith(":45", na=False)
values = df.loc[flt, "extrEndTime"].str.replace(":45", ",5")
values
df.loc[flt, "extrEndTime"] = values

In [27]:
df.extrEndTime = df.extrEndTime.str.replace("19:00 (weekdays), 18:00 (weekends & holidays)", "19")

In [28]:
flt = df.extrStartTime.str.contains("Termin/Uhrzeit", na=False)
values = df.loc[flt, "extrStartTime"].apply(lambda ls: ls.split("\n")[-1]).str.replace(" Uhr", "").str.replace(":00", "").str.replace(":30", ",5")
values
df.loc[flt, "extrStartTime"] = values

In [29]:
df.extrStartTime = df.extrStartTime.str.replace(":00 (for the opening ceremony)", "")

In [30]:
df.extrStartTime.unique()

array(['15', '10', '11', '8', '9', '12', '20', '17', '16', '19,5', '10,5',
       '19', '18', nan, '11:45', '13', '14', '12,5', '11,5', '22', '15,5',
       '13,5', '16,5', '18,5', '17,5', '21', '21,5'], dtype=object)

In [31]:
df.extrEndTime.unique()

array([nan, '10', '19', '17', '16', '15', '12', '20', '11', '19,5', '18',
       '18,5', '13', '16,5', '14', '11,5', '21,5', '22', '23', '20,5',
       '13,5', '22,5', '21'], dtype=object)

In [32]:
df.loc[3156, "extrEndTime"]="18"
df.loc[5495, "extrEndTime"]="18"

### estimate the readiness of the dataset

In [33]:
len(df)

487

In [34]:
df.head()

,Date_(GMT),Job_Link,Origin_URL,Event_URL,Event_name,Job_link_with_screenshot,Start_date_and_end_date,Start_date,Event_description,Event_category,Price,Location_name,Location_URL,Location_details,Location_address,Location_city,Location_ZIP,Location_website,Organizer_URL,Organizer_details,extrStartDate,extrStartTime,extrEndTime,isOnMultipleDays,extrEndDate,extrMultipleDaysDetails,extrCity,extrStreetHouseNumber,extrLocationDetails,extrLocationName
5522,2024-09-14 13:39:58,https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/9fd891e7-b5ad-4d65-b3ba-9f39237662ff,https://sachsen-net.com/veranstaltungen/heute/,https://sachsen-net.com/veranstaltungen/heute/100-Meter-In-30-Minuten-Dresden-118799.html,100 Meter in 30 Minuten,https://dashboard.browse.ai/teams/meslis/robots/2a7ec739-65b9-4453-a18c-ef6311abecda/history/70a6ad3d-be80-4dc9-ba0f-0c09f83b423e,NaN,14.09.2024,"Langsame Fortbewegung im Takt zu elektronischen Klängen als Performance.\nAlle können mitmachen.\n\nTreffpunkt: Am Goldenen Reiter in Dresden.\nSamstag, den 14. September um 15 Uhr.",Verschiedenes,NaN,Hauptstraße,NaN,NaN,NaN,Dresden,NaN,NaN,NaN,NaN,2024-09-14,15,NaN,False,NaN,NaN,Dresden,NaN,NaN,Hauptstraße
6046,2024-09-15 13:38:59,https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/6e4201d7-3cc1-4843-9640-24d74f27776c,https://sachsen-net.com/veranstaltungen/heute/,https://sachsen-net.com/veranstaltungen/heute/12-Dresdner-Familienlauf-Dresden-117753.html,12. Dresdner Familienlauf,https://dashboard.browse.ai/teams/meslis/robots/2a7ec739-65b9-4453-a18c-ef6311abecda/history/1703d87d-511b-432a-a5aa-9addc8db1ad4,NaN,15.09.2024,"Am 15. September 2024 geht es wieder los!\nhttps://www.familienlauf-dresden.de\n\nMacht mit beim 12. Dresdner Familienlauf zugunsten benachteiligter Kinder! Bei uns gibt es gleich drei Highlights für Euch:\n\n1. Laufen und dabei Gutes tun\n2. Sportlich betätigen\n3. Tolle Preise gewinnen\n\nEgal ob jung oder alt – jeder kann mitmachen! Es können auch Firmenteams oder Vereins-, Klassen- bzw. Kindergartengruppen antreten.\n\nEs gibt keine Startgebühr , d.h. es ist ein reiner Spendenlauf. Die Läufer spenden entweder selbst pro 400m (gelaufene Runde) einen selbst gewählten Betrag (z.B. einen Euro) oder suchen sich zuvor Sponsor(en). Sponsoren können z.B. Verwandte, Bekannte, Firmen, Geschäfte, o.ä. sein.\n\nDas erlaufene Geld kommt benachteiligten Kindern zugute, der Spendenzweck variiert jedes Jahr. Unter allen Läufern werden tolle Sachpreise verlost und alle Läufer erhalten eine Teilnehmerurkunde.\n\nGehört der Läufer zu einer Schule, einem Kindergarten oder einem gemeinnützigen Verein, dann gilt das 50:50-Konzept. 50 % der erlaufenen Spenden gehen an UNICEF. Über die anderen 50 % kann die jeweilige Institution entscheiden, ob sie diese ebenfalls UNICEF zukommen lässt oder für eigene Zwecke verwendet.\n\nGehört der Läufer nicht zu einer Schule, einem Kindergarten oder einem gemeinnützigen Verein, dann kommt die gesamte erlaufene Summe UNICEF zugute.",Sportveranstaltungen,NaN,Sportplatz am Spielplatz Holbeinstraße,NaN,NaN,NaN,Dresden,NaN,https://www.familienlauf-dresden.de/,NaN,NaN,2024-09-15,10,10,False,None,NaN,Dresden,NaN,01307 Dresden Holbeinstraße 54,Sportplatz am Spielplatz Holbeinstraße
9873,2024-09-21 13:35:34,https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/9376c504-cb8c-4d46-8b99-dba698389076,https://sachsen-net.com/veranstaltungen/heute/,https://sachsen-net.com/veranstaltungen/heute/15-Comicfest-In-Der-Zentralbibliothek-Dresden-118904.html,15. Comicfest in der Zentralbibliothek,https://dashboard.browse.ai/teams/meslis/robots/2a7ec739-65b9-4453-a18c-ef6311abecda/history/3a64a7e5-9b6d-48ce-a370-ee3d7f4407d0,NaN,21.09.2024,"Am 21. September 2024 feiert die Zentralbibliothek den Comic. Von 10 bis 19 Uhr laden Veranstaltungen, Workshops, Lesungen, Filme, eine Comic-Kunst-Börse und vielen weiteren Angeboten rund um das Thema Com

In [35]:
extracted_columns = []
for col in df.columns:
    if col.startswith("extr"):
        extracted_columns.append(col)
    
extracted_columns.append("isOnMultipleDays")

In [36]:
flt = (df.isOnMultipleDays==False) 
print("total single-day events")
print(len(df[flt]))
for col in extracted_columns:
    flt_2 = df[col].isna()
    print(col+ " has missing rows:")
    print(len(df[flt&flt_2]))

total single-day events
400
extrStartDate has missing rows:
0
extrStartTime has missing rows:
0
extrEndTime has missing rows:
203
extrEndDate has missing rows:
400
extrMultipleDaysDetails has missing rows:
400
extrCity has missing rows:
0
extrStreetHouseNumber has missing rows:
231
extrLocationDetails has missing rows:
224
extrLocationName has missing rows:
13
isOnMultipleDays has missing rows:
0


In [37]:
flt = (df.isOnMultipleDays==True) 
print("total multiple-day events")
print(len(df[flt]))
for col in extracted_columns:
    flt_2 = df[col].isna()
    print(col+ " has missing rows:")
    print(len(df[flt&flt_2]))

total multiple-day events
87
extrStartDate has missing rows:
0
extrStartTime has missing rows:
63
extrEndTime has missing rows:
64
extrEndDate has missing rows:
0
extrMultipleDaysDetails has missing rows:
14
extrCity has missing rows:
0
extrStreetHouseNumber has missing rows:
71
extrLocationDetails has missing rows:
41
extrLocationName has missing rows:
6
isOnMultipleDays has missing rows:
0


In [38]:
#flt = (df.isOnMultipleDays==True) & (df.extrLocationName.isna())  & (df.extrLocationDetails.isna())
#df.loc[flt]

In [39]:
#filename = f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl"
#print(filename)
#df.to_pickle(filename)

# transform multiple events into single events

## single day

In [40]:
df_to_process = df.copy()
df_to_process =  df_to_process.reset_index(names="origin_id")

df_to_process.head()

,origin_id,Date_(GMT),Job_Link,Origin_URL,Event_URL,Event_name,Job_link_with_screenshot,Start_date_and_end_date,Start_date,Event_description,Event_category,Price,Location_name,Location_URL,Location_details,Location_address,Location_city,Location_ZIP,Location_website,Organizer_URL,Organizer_details,extrStartDate,extrStartTime,extrEndTime,isOnMultipleDays,extrEndDate,extrMultipleDaysDetails,extrCity,extrStreetHouseNumber,extrLocationDetails,extrLocationName
0,5522,2024-09-14 13:39:58,https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/9fd891e7-b5ad-4d65-b3ba-9f39237662ff,https://sachsen-net.com/veranstaltungen/heute/,https://sachsen-net.com/veranstaltungen/heute/100-Meter-In-30-Minuten-Dresden-118799.html,100 Meter in 30 Minuten,https://dashboard.browse.ai/teams/meslis/robots/2a7ec739-65b9-4453-a18c-ef6311abecda/history/70a6ad3d-be80-4dc9-ba0f-0c09f83b423e,NaN,14.09.2024,"Langsame Fortbewegung im Takt zu elektronischen Klängen als Performance.\nAlle können mitmachen.\n\nTreffpunkt: Am Goldenen Reiter in Dresden.\nSamstag, den 14. September um 15 Uhr.",Verschiedenes,NaN,Hauptstraße,NaN,NaN,NaN,Dresden,NaN,NaN,NaN,NaN,2024-09-14,15,NaN,False,NaN,NaN,Dresden,NaN,NaN,Hauptstraße
1,6046,2024-09-15 13:38:59,https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/6e4201d7-3cc1-4843-9640-24d74f27776c,https://sachsen-net.com/veranstaltungen/heute/,https://sachsen-net.com/veranstaltungen/heute/12-Dresdner-Familienlauf-Dresden-117753.html,12. Dresdner Familienlauf,https://dashboard.browse.ai/teams/meslis/robots/2a7ec739-65b9-4453-a18c-ef6311abecda/history/1703d87d-511b-432a-a5aa-9addc8db1ad4,NaN,15.09.2024,"Am 15. September 2024 geht es wieder los!\nhttps://www.familienlauf-dresden.de\n\nMacht mit beim 12. Dresdner Familienlauf zugunsten benachteiligter Kinder! Bei uns gibt es gleich drei Highlights für Euch:\n\n1. Laufen und dabei Gutes tun\n2. Sportlich betätigen\n3. Tolle Preise gewinnen\n\nEgal ob jung oder alt – jeder kann mitmachen! Es können auch Firmenteams oder Vereins-, Klassen- bzw. Kindergartengruppen antreten.\n\nEs gibt keine Startgebühr , d.h. es ist ein reiner Spendenlauf. Die Läufer spenden entweder selbst pro 400m (gelaufene Runde) einen selbst gewählten Betrag (z.B. einen Euro) oder suchen sich zuvor Sponsor(en). Sponsoren können z.B. Verwandte, Bekannte, Firmen, Geschäfte, o.ä. sein.\n\nDas erlaufene Geld kommt benachteiligten Kindern zugute, der Spendenzweck variiert jedes Jahr. Unter allen Läufern werden tolle Sachpreise verlost und alle Läufer erhalten eine Teilnehmerurkunde.\n\nGehört der Läufer zu einer Schule, einem Kindergarten oder einem gemeinnützigen Verein, dann gilt das 50:50-Konzept. 50 % der erlaufenen Spenden gehen an UNICEF. Über die anderen 50 % kann die jeweilige Institution entscheiden, ob sie diese ebenfalls UNICEF zukommen lässt oder für eigene Zwecke verwendet.\n\nGehört der Läufer nicht zu einer Schule, einem Kindergarten oder einem gemeinnützigen Verein, dann kommt die gesamte erlaufene Summe UNICEF zugute.",Sportveranstaltungen,NaN,Sportplatz am Spielplatz Holbeinstraße,NaN,NaN,NaN,Dresden,NaN,https://www.familienlauf-dresden.de/,NaN,NaN,2024-09-15,10,10,False,None,NaN,Dresden,NaN,01307 Dresden Holbeinstraße 54,Sportplatz am Spielplatz Holbeinstraße
2,9873,2024-09-21 13:35:34,https://dashboard.browse.ai/teams/meslis/robots/7afcec8c-6cf5-48ab-8bfe-561611c64908/history/9376c504-cb8c-4d46-8b99-dba698389076,https://sachsen-net.com/veranstaltungen/heute/,https://sachsen-net.com/veranstaltungen/heute/15-Comicfest-In-Der-Zentralbibliothek-Dresden-118904.html,15. Comicfest in der Zentralbibliothek,https://dashboard.browse.ai/teams/meslis/robots/2a7ec739-65b9-4453-a18c-ef6311abecda/history/3a64a7e5-9b6d-48ce-a370-ee3d7f4407d0,NaN,21.09.2024,"Am 21. September 2024 feiert die Zentralbibliothek den Comic. Von 10 bis 19 Uhr laden Veranstaltungen, Workshops, Lesungen, Filme, eine Comic-Kunst-Börse und vielen weiteren Angeboten rund 

In [41]:
cols_to_keep = extracted_columns.copy()
cols_to_keep.append("origin_id")
cols_to_keep.append("Event_category")


In [42]:
df_to_process = df_to_process[cols_to_keep]

In [43]:
df_to_process.head()

,extrStartDate,extrStartTime,extrEndTime,extrEndDate,extrMultipleDaysDetails,extrCity,extrStreetHouseNumber,extrLocationDetails,extrLocationName,isOnMultipleDays,origin_id,Event_category
0,2024-09-14,15,NaN,NaN,NaN,Dresden,NaN,NaN,Hauptstraße,False,5522,Verschiedenes
1,2024-09-15,10,10,None,NaN,Dresden,NaN,01307 Dresden Holbeinstraße 54,Sportplatz am Spielplatz Holbeinstraße,False,6046,Sportveranstaltungen
2,2024-09-21,10,19,NaN,NaN,Dresden,NaN,NaN,Zentralbibliothek,False,9873,Verschiedenes
3,2024-09-29,11,17,NaN,NaN,Dresden,NaN,NaN,Fährgarten Johannstadt,False,14288,"Feste, Märkte & Messen"
4,2024-10-20,8,16,NaN,NaN,Dresden,NaN,NaN,Alberthafen,False,25938,"Feste, Märkte & Messen"


In [44]:
len(df_to_process)

487

In [45]:
all_cols_except_id = df_to_process.columns.tolist()
all_cols_except_id.remove("origin_id")

In [46]:
df_to_process.drop_duplicates(inplace=True, subset=all_cols_except_id)
len(df_to_process)

477

In [47]:
df_to_process.columns

Index(['extrStartDate', 'extrStartTime', 'extrEndTime', 'extrEndDate',
       'extrMultipleDaysDetails', 'extrCity', 'extrStreetHouseNumber',
       'extrLocationDetails', 'extrLocationName', 'isOnMultipleDays',
       'origin_id', 'Event_category'],
      dtype='object')

In [48]:
flt = df_to_process.isOnMultipleDays==False
df_final = df_to_process[flt].copy()
df_to_process = df_to_process[~flt]


In [49]:
df_to_process.head()

,extrStartDate,extrStartTime,extrEndTime,extrEndDate,extrMultipleDaysDetails,extrCity,extrStreetHouseNumber,extrLocationDetails,extrLocationName,isOnMultipleDays,origin_id,Event_category
8,2024-09-21,9,17,2024-09-22,NaN,Dresden,NaN,01069 Dresden Stallhof,Stallhof Dresden,True,9865,Ausstellungen & Museen
20,2024-09-28,10,18,2024-09-29,NaN,Dresden,NaN,01326 Dresden-Pillnitz August-Böckstiegel-Straße,Schlosspark Pillnitz,True,13859,Ausstellungen & Museen
36,2024-10-03,NaN,NaN,2024-10-06,"Thu: 10:00-20:00, Fri: 11:00-19:00, Sat: 10:00-20:00, Sun: 10:00-20:00",Dresden,NaN,01465 Dresden-Langebrück Gänsefuß 55,Hofewiese,True,16052,"Feste, Märkte & Messen"
41,2024-09-13,NaN,NaN,2024-10-06,"Sun-Thu: 10:00-19:00, Fri-Sat & public holiday: 10:00-20:00, Opening day (13.09): 12:00-20:00",Dresden,NaN,01069 Dresden Altmarkt,Altmarkt,True,4937,"Feste, Märkte & Messen"
42,2024-09-14,15,18,2024-09-15,NaN,Dresden,NaN,01067 Dresden Neumarkt,Altstadt,True,5495,Verschiedenes


## with existing start and end time

In [50]:
flt = (~df_to_process.extrStartDate.isna()) & (~df_to_process.extrEndDate.isna()) & (~df_to_process.extrStartTime.isna()) & (~df_to_process.extrEndTime.isna())
df_tmp = df_to_process[flt].copy()
df_to_process = df_to_process[~flt]
df_tmp
# expand multiple day events to single day events
df_tmp["extrStartDate"] = pd.to_datetime(df_tmp["extrStartDate"])
df_tmp["extrEndDate"] = pd.to_datetime(df_tmp["extrEndDate"])
df_tmp


,extrStartDate,extrStartTime,extrEndTime,extrEndDate,extrMultipleDaysDetails,extrCity,extrStreetHouseNumber,extrLocationDetails,extrLocationName,isOnMultipleDays,origin_id,Event_category
8,2024-09-21,9,17,2024-09-22,NaN,Dresden,NaN,01069 Dresden Stallhof,Stallhof Dresden,True,9865,Ausstellungen & Museen
20,2024-09-28,10,18,2024-09-29,NaN,Dresden,NaN,01326 Dresden-Pillnitz August-Böckstiegel-Straße,Schlosspark Pillnitz,True,13859,Ausstellungen & Museen
42,2024-09-14,15,18,2024-09-15,NaN,Dresden,NaN,01067 Dresden Neumarkt,Altstadt,True,5495,Verschiedenes
46,2024-10-07,15,"16,5",2024-10-10,NaN,Dresden,NaN,01109 Dresden Wilhelmine-Reichard-Ring 1,Flughafen Dresden,True,18582,Verschiedenes
47,2024-10-14,15,"16,5",2024-10-17,NaN,Dresden,NaN,01109 Dresden Wilhelmine-Reichard-Ring 1,Flughafen Dresden,True,22687,Verschiedenes
49,2024-06-01,10,17,2024-09-29,NaN,Dresden,NaN,01326 Dresden-Pillnitz August-Böckstiegel-Straße,Schloss Pillnitz,True,3592,Ausstellungen & Museen
79,2024-10-10,9,17,2024-10-12,NaN,Dresden,NaN,01067 Dresden Messering 6,Messe Dresden,True,22195,"Feste, Märkte & Messen"
143,2024-09-13,20,23,2024-09-14,NaN,Dresden,NaN,01069 Dresden Hbf. an der Gläsernen Manufaktur,Parkeisenbahn Dresden,True,4931,Verschiedenes
163,2024-10-18,13,"20,5",2024-10-20,NaN,Dresden,NaN,01097 Dresden Eisenbahnstraße 1,Blaue Fabrik e.V.,True,24499,Verschiedenes
213,2024-09-20,11,18,2024-09-21,NaN,Dresden,NaN,01139 Dresden Peschelstraße 39,ELBEPARK Dresden,True,9282,Verschiedenes


In [51]:
rows_to_append = []
for i, row in df_tmp.iterrows():
    start_date = row["extrStartDate"]
    end_date = row["extrEndDate"]
    print(f"{start_date=}")
    print(f"{end_date=}")
    date_range = pd.date_range(start_date, end_date)
    
    for date in date_range: # inclusive range
        row_to_append = row.copy()
        row_to_append["extrStartDate"] = date.strftime('%Y-%m-%d')
        rows_to_append.append(row_to_append)
        


start_date=Timestamp('2024-09-21 00:00:00')
end_date=Timestamp('2024-09-22 00:00:00')
start_date=Timestamp('2024-09-28 00:00:00')
end_date=Timestamp('2024-09-29 00:00:00')
start_date=Timestamp('2024-09-14 00:00:00')
end_date=Timestamp('2024-09-15 00:00:00')
start_date=Timestamp('2024-10-07 00:00:00')
end_date=Timestamp('2024-10-10 00:00:00')
start_date=Timestamp('2024-10-14 00:00:00')
end_date=Timestamp('2024-10-17 00:00:00')
start_date=Timestamp('2024-06-01 00:00:00')
end_date=Timestamp('2024-09-29 00:00:00')
start_date=Timestamp('2024-10-10 00:00:00')
end_date=Timestamp('2024-10-12 00:00:00')
start_date=Timestamp('2024-09-13 00:00:00')
end_date=Timestamp('2024-09-14 00:00:00')
start_date=Timestamp('2024-10-18 00:00:00')
end_date=Timestamp('2024-10-20 00:00:00')
start_date=Timestamp('2024-09-20 00:00:00')
end_date=Timestamp('2024-09-21 00:00:00')
start_date=Timestamp('2024-09-01 00:00:00')
end_date=Timestamp('2024-09-29 00:00:00')
start_date=Timestamp('2024-09-13 00:00:00')
end_date=T

In [52]:
df_to_append = pd.DataFrame(rows_to_append)

In [53]:
df_final = pd.concat([df_final, df_to_append])

/var/folders/zz/b00th7c11zqbbg9m2fhn24d40000gp/T/ipykernel_10563/2254076344.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, df_to_append])


## with missing time

In [54]:
with open("../data/keys/OpenAIKey (vkalonova).txt", "r") as file:
        openai_api_key = file.read().strip()

In [55]:
client = OpenAI(api_key=openai_api_key)

In [56]:
system_content = """ Give me a record of date, start and end times of events for each date in which an event happens. Taake weekdays onto account.
Example input:
2024-10-03 till 2024-10-06
MultipleDaysDetails = Thu: 10:00-20:00, Fri: 11:00-19:00, Sat: 10:00-20:00, Sun: 10:00-20:00
Example output:
2024-10-03, 10:00-20:00
2024-10-04, 11:00,-19:00
2024-10-05, 10:00-20:00
2024-10-06, 10:00-20:00
"""

In [57]:
id = 41
prompt = f""" {df_to_process.loc[id, "extrStartDate"]} till {df_to_process.loc[id, "extrEndDate"]}
MultipleDaysDetails = {df_to_process.loc[id, "extrMultipleDaysDetails"]}
"""
prompt

' 2024-09-13 till 2024-10-06\nMultipleDaysDetails = Sun-Thu: 10:00-19:00, Fri-Sat & public holiday: 10:00-20:00, Opening day (13.09): 12:00-20:00\n'

In [58]:
# completion = client.chat.completions.create(
#     model="gpt-4o",
#     messages=[
#         {"role": "system", "content": system_content},
#         {
#             "role": "user",
#             "content": prompt
#         }
#     ]
# )



In [59]:
# openai_respones = []
# openai_respones.append({ "id": 41, "response": completion.choices[0].message.content})


In [60]:
# for i, row in df_to_process.iterrows():
#     print(i)
#     print()
#     if i==41:
#         continue

#     else:
#         prompt = f""" {df_to_process.loc[i, "extrStartDate"]} till {df_to_process.loc[i, "extrEndDate"]}
# MultipleDaysDetails = {df_to_process.loc[i, "extrMultipleDaysDetails"]}"""
#         print(prompt)
#         completion = client.chat.completions.create(
#             model="gpt-4o",
#             messages=[
#                 {"role": "system", "content": system_content},
#                 {
#                     "role": "user",
#                     "content": prompt
#                 }
#             ]
#         )
#         res = completion.choices[0].message.content
#         print(res)

#         openai_respones.append({ "id": i, "response": completion.choices[0].message.content})

        



In [61]:
df_openai_responses = pd.read_csv(f"{DATA_FOLDER}df_openai_responses_for_multiple_days_events 2025-01-03.csv")

In [62]:
# df_openai_responses = pd.DataFrame(openai_respones)
df_openai_responses.head()


,id,response,processed_response
0,41,"2024-09-13, 12:00-20:00 \n2024-09-14, 10:00-20:00 \n2024-09-15, 10:00-19:00 \n2024-09-16, 10:00-19:00 \n2024-09-17, 10:00-19:00 \n2024-09-18, 10:00-19:00 \n2024-09-19, 10:00-19:00 \n2024-09-20, 10:00-20:00 \n2024-09-21, 10:00-20:00 \n2024-09-22, 10:00-19:00 \n2024-09-23, 10:00-19:00 \n2024-09-24, 10:00-19:00 \n2024-09-25, 10:00-19:00 \n2024-09-26, 10:00-19:00 \n2024-09-27, 10:00-20:00 \n2024-09-28, 10:00-20:00 \n2024-09-29, 10:00-19:00 \n2024-09-30, 10:00-19:00 \n2024-10-01, 10:00-19:00 \n2024-10-02, 10:00-19:00 \n2024-10-03, 10:00-19:00 \n2024-10-04, 10:00-20:00 \n2024-10-05, 10:00-20:00 \n2024-10-06, 10:00-19:00","['2024-09-13, 12:00-20:00 ', '2024-09-14, 10:00-20:00 ', '2024-09-15, 10:00-19:00 ', '2024-09-16, 10:00-19:00 ', '2024-09-17, 10:00-19:00 ', '2024-09-18, 10:00-19:00 ', '2024-09-19, 10:00-19:00 ', '2024-09-20, 10:00-20:00 ', '2024-09-21, 10:00-20:00 ', '2024-09-22, 10:00-19:00 ', '2024-09-23, 10:00-19:00 ', '2024-09-24, 10:00-19:00 ', '2024-09-25, 10:00-19:00 ', '2024-09-26, 10:00-19:00 ', '2024-09-27, 10:00-20:00 ', '2024-09-28, 10:00-20:00 ', '2024-09-29, 10:00-19:00 ', '2024-09-30, 10:00-19:00 ', '2024-10-01, 10:00-19:00 ', '2024-10-02, 10:00-19:00 ', '2024-10-03, 10:00-19:00 ', '2024-10-04, 10:00-20:00 ', '2024-10-05, 10:00-20:00 ', '2024-10-06, 10:00-19:00 ']"
1,36,"2024-10-03, 10:00-20:00 \n2024-10-04, 11:00-19:00 \n2024-10-05, 10:00-20:00 \n2024-10-06, 10:00-20:00","['2024-10-03, 10:00-20:00 ', '2024-10-04, 11:00-19:00 ', '2024-10-05, 10:00-20:00 ', '2024-10-06, 10:00-20:00']"
2,43,"2024-10-25, 10:00-19:00, 19:00-Closing \n2024-10-26, 10:00-18:00 \n2024-10-27, 10:00-18:00 \n2024-10-29, 10:00-18:00 \n2024-10-30, 10:00-18:00 \n2024-10-31, 10:00-18:00","['2024-10-25, 10:00-19:00, 19:00-Closing ', '2024-10-26, 10:00-18:00 ', '2024-10-27, 10:00-18:00 ', '2024-10-29, 10:00-18:00 ', '2024-10-30, 10:00-18:00 ', '2024-10-31, 10:00-18:00 ']"
3,44,"2024-10-26, 13:00-18:00, 19:30-22:30\n2024-10-27, 10:00-18:00\n2024-10-28, 10:00-15:00, 20:00-22:00\n2024-10-29, 10:00-15:00, 19:30-21:30\n2024-10-30, 10:00-15:00, 20:00-22:00\n2024-10-31, 10:00-15:00, 19:30-21:30","['2024-10-26, 13:00-18:00, 19:30-22:30', '2024-10-27, 10:00-18:00', '2024-10-28, 10:00-15:00, 20:00-22:00', '2024-10-29, 10:00-15:00, 19:30-21:30', '2024-10-30, 10:00-15:00, 20:00-22:00', '2024-10-31, 10:00-15:00, 19:30-21:30']"
4,52,"2024-10-05, 10:30-12:00, 13:30-15:00, 16:30\n2024-10-06, 10:30-12:00, 13:30-15:00, 16:30\n2024-10-08, 09:30-11:00\n2024-10-09, 09:30-11:00\n2024-10-10, 09:30-11:00\n2024-10-11, 09:30-11:00, 12:30-14:00, 15:30\n2024-10-12, 10:30-12:00, 13:30-15:00, 16:30\n2024-10-13, 10:30-12:00, 13:30-15:00, 16:30\n2024-10-15, 09:30-11:00\n2024-10-16, 09:30-11:00\n2024-10-17, 09:30-11:00\n2024-10-18, 09:30-11:00, 12:30-14:00, 15:30\n2024-10-19, 10:30-12:00, 13:30-15:00, 16:30\n2024-10-20, 10:30-12:00, 13:30-15:00, 16:30","['2024-10-05, 10:30-12:00, 13:30-15:00, 16:30', '2024-10-06, 10:30-12:00, 13:30-15:00, 16:30', '2024-10-08, 09:30-11:00', '2024-10-09, 09:30-11:00', '2024-10-10, 09:30-11:00', '2024-10-11, 09:30-11:00, 12:30-14:00, 15:30', '2024-10-12, 10:30-12:00, 13:30-15:00, 16:30', '2024-10-13, 10:30-12:00, 13:30-15:00, 16:30', '2024-10-15, 09:30-11:00', '2024-10-16, 09:30-11:00', '2024-10-17, 09:30-11:00', '2024-10-18, 09:30-11:00, 12:30-14:00, 15:30', '2024-10-19, 10:30-12:00, 13:30-15:00, 16:30', '2024-10-20, 10:30-12:00, 13:30-15:00, 16:30']"


In [63]:
df_openai_responses["processed_response"] = df_openai_responses.response.apply(lambda x: x.split("\n")).apply(lambda x: [i for i in x if "2024" in i])

In [64]:
df_openai_responses.head()

,id,response,processed_response
0,41,"2024-09-13, 12:00-20:00 \n2024-09-14, 10:00-20:00 \n2024-09-15, 10:00-19:00 \n2024-09-16, 10:00-19:00 \n2024-09-17, 10:00-19:00 \n2024-09-18, 10:00-19:00 \n2024-09-19, 10:00-19:00 \n2024-09-20, 10:00-20:00 \n2024-09-21, 10:00-20:00 \n2024-09-22, 10:00-19:00 \n2024-09-23, 10:00-19:00 \n2024-09-24, 10:00-19:00 \n2024-09-25, 10:00-19:00 \n2024-09-26, 10:00-19:00 \n2024-09-27, 10:00-20:00 \n2024-09-28, 10:00-20:00 \n2024-09-29, 10:00-19:00 \n2024-09-30, 10:00-19:00 \n2024-10-01, 10:00-19:00 \n2024-10-02, 10:00-19:00 \n2024-10-03, 10:00-19:00 \n2024-10-04, 10:00-20:00 \n2024-10-05, 10:00-20:00 \n2024-10-06, 10:00-19:00","[2024-09-13, 12:00-20:00 , 2024-09-14, 10:00-20:00 , 2024-09-15, 10:00-19:00 , 2024-09-16, 10:00-19:00 , 2024-09-17, 10:00-19:00 , 2024-09-18, 10:00-19:00 , 2024-09-19, 10:00-19:00 , 2024-09-20, 10:00-20:00 , 2024-09-21, 10:00-20:00 , 2024-09-22, 10:00-19:00 , 2024-09-23, 10:00-19:00 , 2024-09-24, 10:00-19:00 , 2024-09-25, 10:00-19:00 , 2024-09-26, 10:00-19:00 , 2024-09-27, 10:00-20:00 , 2024-09-28, 10:00-20:00 , 2024-09-29, 10:00-19:00 , 2024-09-30, 10:00-19:00 , 2024-10-01, 10:00-19:00 , 2024-10-02, 10:00-19:00 , 2024-10-03, 10:00-19:00 , 2024-10-04, 10:00-20:00 , 2024-10-05, 10:00-20:00 , 2024-10-06, 10:00-19:00 ]"
1,36,"2024-10-03, 10:00-20:00 \n2024-10-04, 11:00-19:00 \n2024-10-05, 10:00-20:00 \n2024-10-06, 10:00-20:00","[2024-10-03, 10:00-20:00 , 2024-10-04, 11:00-19:00 , 2024-10-05, 10:00-20:00 , 2024-10-06, 10:00-20:00]"
2,43,"2024-10-25, 10:00-19:00, 19:00-Closing \n2024-10-26, 10:00-18:00 \n2024-10-27, 10:00-18:00 \n2024-10-29, 10:00-18:00 \n2024-10-30, 10:00-18:00 \n2024-10-31, 10:00-18:00","[2024-10-25, 10:00-19:00, 19:00-Closing , 2024-10-26, 10:00-18:00 , 2024-10-27, 10:00-18:00 , 2024-10-29, 10:00-18:00 , 2024-10-30, 10:00-18:00 , 2024-10-31, 10:00-18:00 ]"
3,44,"2024-10-26, 13:00-18:00, 19:30-22:30\n2024-10-27, 10:00-18:00\n2024-10-28, 10:00-15:00, 20:00-22:00\n2024-10-29, 10:00-15:00, 19:30-21:30\n2024-10-30, 10:00-15:00, 20:00-22:00\n2024-10-31, 10:00-15:00, 19:30-21:30","[2024-10-26, 13:00-18:00, 19:30-22:30, 2024-10-27, 10:00-18:00, 2024-10-28, 10:00-15:00, 20:00-22:00, 2024-10-29, 10:00-15:00, 19:30-21:30, 2024-10-30, 10:00-15:00, 20:00-22:00, 2024-10-31, 10:00-15:00, 19:30-21:30]"
4,52,"2024-10-05, 10:30-12:00, 13:30-15:00, 16:30\n2024-10-06, 10:30-12:00, 13:30-15:00, 16:30\n2024-10-08, 09:30-11:00\n2024-10-09, 09:30-11:00\n2024-10-10, 09:30-11:00\n2024-10-11, 09:30-11:00, 12:30-14:00, 15:30\n2024-10-12, 10:30-12:00, 13:30-15:00, 16:30\n2024-10-13, 10:30-12:00, 13:30-15:00, 16:30\n2024-10-15, 09:30-11:00\n2024-10-16, 09:30-11:00\n2024-10-17, 09:30-11:00\n2024-10-18, 09:30-11:00, 12:30-14:00, 15:30\n2024-10-19, 10:30-12:00, 13:30-15:00, 16:30\n2024-10-20, 10:30-12:00, 13:30-15:00, 16:30","[2024-10-05, 10:30-12:00, 13:30-15:00, 16:30, 2024-10-06, 10:30-12:00, 13:30-15:00, 16:30, 2024-10-08, 09:30-11:00, 2024-10-09, 09:30-11:00, 2024-10-10, 09:30-11:00, 2024-10-11, 09:30-11:00, 12:30-14:00, 15:30, 2024-10-12, 10:30-12:00, 13:30-15:00, 16:30, 2024-10-13, 10:30-12:00, 13:30-15:00, 16:30, 2024-10-15, 09:30-11:00, 2024-10-16, 09:30-11:00, 2024-10-17, 09:30-11:00, 2024-10-18, 09:30-11:00, 12:30-14:00, 15:30, 2024-10-19, 10:30-12:00, 13:30-15:00, 16:30, 2024-10-20, 10:30-12:00, 13:30-15:00, 16:30]"


### process openai responses

In [65]:
rows_to_append = []
skipped_rows = 0
openai_explanations = ["Now let's", "To accurately provide", "We need to consider", "To provide a schedule", "Continuing in this ",
                       "Identify", "Check", "To determine", "map these details", "Given the provided input", "If the day is", "To create a schedule",
                       "Note that", "To provide"]
closed_events_escriptions = ["Closed", "No event", "geschlossen", "No event"]
holiday_descriptions = ["Thursday - Tag der Deutschen Einheit", " (Holiday - Labor Day)", 
                        "Thursday, Halloween; not a public holiday", "German Unity Day, public holiday",
                        "Feiertag in Deutschland: Tag der Deutschen Einheit", "Tag der Deutschen Einheit", "(", ")"
                        ]
for _, openai_row in df_openai_responses.iterrows():
    single_dates_responses = openai_row["response"].split("\n")
    # only keep elemets in a list with at least 1 digit

    for res in single_dates_responses:
        res = res.replace("**", "")
        for full_german_weekday in [", Montag, ", ", Dienstag, ", ", Mittwoch, ", ", Donnerstag, ", ", Freitag, ", ", Samstag, ", ", Sonntag, "]:
                res = res.replace(full_german_weekday, ", ")
        for full_german_weekday in [" (Monday): ", " (Tuesday): ", " (Wednesday): ", " (Thursday): ", " (Friday): ", " (Saturday): ", " (Sunday): "]:
                res = res.replace(full_german_weekday, ", ")

        
        """
        for full_weekday in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]:
            res = res.replace(full_weekday, "")
        
        for weekday in ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]:
            res = res.replace(weekday, "")
        res = res.replace(", , ", ", ")
        res = res.replace(" (): ", ", ") """
        for txt in holiday_descriptions:
            res = res.replace(txt, "")
        res = res.replace("  ", " ")

        skip_outer_loop = False
        if not re.search(r"\d", res):
            continue
        for txt in openai_explanations:
            if txt in res:
                skip_outer_loop=True
                break 
        for txt in closed_events_escriptions:
            if txt in res:
                skip_outer_loop=True
                break

        if skip_outer_loop:
            continue
        try:
            date, times1, times2, times3 = res.split(", ")
            start_time1, end_time1 = times1.split("-")
            start_time1 = start_time1.strip().replace(":00", "").replace(":30", ",5")
            end_time1 = end_time1.strip().replace(":00", "").replace(":30", ",5")
            start_time2, end_time2 = times2.split("-")
            start_time2 = start_time2.strip().replace(":00", "").replace(":30", ",5")
            end_time2 = end_time2.strip().replace(":00", "").replace(":30", ",5")
            start_time3 = times3
            end_time3 = None
            id = openai_row["id"]
            assert len(df_to_process[df_to_process.index == id]) == 1
            row_to_append = df_to_process.loc[id].copy()
            row_to_append["extrStartDate"] = date
            #row_to_append["extrEndDate"] = None
            row_to_append["extrStartTime"] = start_time1
            row_to_append["extrEndTime"] = end_time1
            rows_to_append.append(row_to_append)
            row_to_append = df_to_process.loc[id].copy()
            row_to_append["extrStartDate"] = date
            #row_to_append["extrEndDate"] = None 
            row_to_append["extrStartTime"] = start_time2
            row_to_append["extrEndTime"] = end_time2
            rows_to_append.append(row_to_append)
            row_to_append = df_to_process.loc[id].copy()
            row_to_append["extrStartDate"] = date
            #row_to_append["extrEndDate"] = None
            row_to_append["extrStartTime"] = start_time3
            row_to_append["extrEndTime"] = end_time3
            rows_to_append.append(row_to_append)
        except Exception as e:            
            try:
                try:    
                    date, times1, times2 = res.split(", ")
                    start_time1, end_time1 = times1.split("-")
                    start_time1 = start_time1.strip().replace(":00", "").replace(":30", ",5")
                    end_time1 = end_time1.strip().replace(":00", "").replace(":30", ",5")
                    start_time2, end_time2 = times2.split("-")
                    start_time2 = start_time2.strip().replace(":00", "").replace(":30", ",5")
                    end_time2 = end_time2.strip().replace(":00", "").replace(":30", ",5")
                    id = openai_row["id"]
                    assert len(df_to_process[df_to_process.index == id]) == 1
                    row_to_append = df_to_process.loc[id].copy()
                    row_to_append["extrStartDate"] = date
                    #row_to_append["extrEndDate"] = None
                    row_to_append["extrStartTime"] = start_time1
                    row_to_append["extrEndTime"] = end_time1
                    rows_to_append.append(row_to_append)
                    row_to_append = df_to_process.loc[id].copy()
                    row_to_append["extrStartDate"] = date
                    #row_to_append["extrEndDate"] = None
                    row_to_append["extrStartTime"] = start_time2
                    row_to_append["extrEndTime"] = end_time2
                    rows_to_append.append(row_to_append)

                # write a regexp for dates like this October 1, 2024, 10:00-18:00
                #if re.search(res, r"\d{4}-\d{2}-\d{2}"):

                except Exception as e:
                    date, times = res.split(", ")
                    #print(f"{date=}")
                    #print(f"{times=}")
                    start_time, end_time = times.split("-")
                    start_time = start_time.strip().replace(":00", "").replace(":30", ",5")
                    end_time = end_time.strip().replace(":00", "").replace(":30", ",5")
                    #print(f"{start_time=}")
                    #print(f"{end_time=}")
                    id = openai_row["id"]
                    #print(f"{id=}")
                    assert len(df_to_process[df_to_process.index == id]) == 1
                    row_to_append = df_to_process.loc[id].copy()
                    #print(f"{row_to_append=}")
                    row_to_append["extrStartDate"] = date
                    #row_to_append["extrEndDate"] = None
                    row_to_append["extrStartTime"] = start_time
                    row_to_append["extrEndTime"] = end_time
                    rows_to_append.append(row_to_append)
            except Exception as e:
                print(f"could not process {res} because of {e}")
                skipped_rows += 1
                print(f"extrMultipleDaysDetails: {   df_to_process.loc[id, 'extrMultipleDaysDetails']}")
                print()




could not process - Monday to Friday Di - Fr: 09:00-17:00 because of not enough values to unpack (expected 2, got 1)
extrMultipleDaysDetails: Fri: 18:30-20:30, Sat: 09:00-16:00

could not process - Saturday and Sunday Sa/So: 10:00-18:00 because of not enough values to unpack (expected 2, got 1)
extrMultipleDaysDetails: Fri: 18:30-20:30, Sat: 09:00-16:00

could not process - Tuesday to Friday: 09:00-17:00 because of not enough values to unpack (expected 2, got 1)
extrMultipleDaysDetails: Mon-Fri: 10:00-17:00, Sat-Sun, Holidays: 10:00-18:00

could not process - Saturday/Sunday/holidays: 10:00-18:00 because of not enough values to unpack (expected 2, got 1)
extrMultipleDaysDetails: Mon-Fri: 10:00-17:00, Sat-Sun, Holidays: 10:00-18:00

could not process - 2024-09-13, Eröffnungstag von 12 bis 20 Uhr because of not enough values to unpack (expected 2, got 1)
extrMultipleDaysDetails: Wed-Fri: 10:00-17:00, Sat-Sun: 12:00-17:00

could not process - 2024-10-03 : 10:00-20:00 because of not enough

In [66]:
len(rows_to_append)

1498

In [67]:
skipped_rows

63

In [68]:
df_to_append =  pd.DataFrame(rows_to_append)

In [69]:
df_to_append.extrEndTime.unique()

array(['20', '19', 'Closing', '18', '22,5', '15', '22', '21,5', '12',
       None, '11', '14', '17', '19,5', '21', '20,5', '16', '', '23',
       '18 Sunday', '19 Wednesday', '19 Thursday', '19 Friday',
       '18 Saturday'], dtype=object)

In [70]:
df_to_append.extrStartTime.unique()

array(['12', '10', '11', '19', '13', '19,5', '20', '10,5', '13,5',
       '16:30', '09,5', '12,5', '15:30', '15', '00', '16', '14', '18,5',
       '09', '17', '18', 'Feiertag', 'including holidays from 12',
       'Wed – 11', 'Thu – 11', 'Fri – 11', 'Sat – 11', 'SO Sonntag: 10'],
      dtype=object)

In [71]:
flt = df_to_append.extrStartDate.str.startswith("- ")
values = df_to_append.loc[flt, "extrStartDate"].str.replace("- ", "")
df_to_append.loc[flt, "extrStartDate"] = values

In [72]:
for weekday in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]:
    df_to_append.extrStartDate = df_to_append.extrStartDate.str.replace(weekday, "")
    df_to_append.extrEndTime = df_to_append.extrEndTime.str.replace(weekday, "")


In [73]:
df_to_append.extrStartDate =  df_to_append.extrStartDate.str.strip()

In [74]:
df_to_append.extrStartDate.unique()

array(['2024-09-13', '2024-09-14', '2024-09-15', '2024-09-16',
       '2024-09-17', '2024-09-18', '2024-09-19', '2024-09-20',
       '2024-09-21', '2024-09-22', '2024-09-23', '2024-09-24',
       '2024-09-25', '2024-09-26', '2024-09-27', '2024-09-28',
       '2024-09-29', '2024-09-30', '2024-10-01', '2024-10-02',
       '2024-10-03', '2024-10-04', '2024-10-05', '2024-10-06',
       '2024-10-25', '2024-10-26', '2024-10-27', '2024-10-29',
       '2024-10-30', '2024-10-31', '2024-10-28', '2024-10-08',
       '2024-10-09', '2024-10-10', '2024-10-11', '2024-10-12',
       '2024-10-13', '2024-10-15', '2024-10-16', '2024-10-17',
       '2024-10-18', '2024-10-19', '2024-10-20', '2024-08-29',
       '2024-08-30', '2024-08-31', '2024-09-01', '2024-09-02',
       '2024-09-03', '2024-09-04', '2024-09-05', '2024-09-06',
       '2024-09-07', '2024-09-08', '2024-09-09', '2024-09-10',
       '2024-09-11', '2024-09-12', '2024-10-07', '2024-10-22',
       '2024-10-23', '2024-10-24', '2024-10-21', '2024-

In [75]:
flt = df_to_append.extrStartDate.str.contains("2024", na=False)
df_to_append = df_to_append[flt]


In [76]:
len(df_to_append)

1462

In [77]:
df_to_append.extrStartTime = df_to_append.extrStartTime.str.replace(":30", ",5")

In [78]:
flt = df_to_append.extrStartTime.str.contains("Feiertag")
df_to_append = df_to_append[~flt]

In [79]:
df_to_append.extrStartTime.value_counts()

extrStartTime
10      714
09      280
12      167
14      119
16       51
11       26
13       24
10,5     23
15       19
09,5      8
16,5      6
13,5      6
00        4
19,5      3
15,5      2
20        2
12,5      2
17        2
19        1
18,5      1
18        1
Name: count, dtype: int64

In [80]:
df_to_append.extrEndTime.value_counts()

extrEndTime
17         616
18         575
19          68
18          34
22          33
19          24
21          23
20          23
23          21
15          11
11           8
12           6
21,5         2
19,5         2
14           2
Closing      1
22,5         1
20,5         1
16           1
             1
Name: count, dtype: int64

In [81]:
flt = df_to_append.extrEndTime.str.contains("Closing", na=False)
df_to_append = df_to_append[~flt]

In [82]:
len(df_to_append)

1460

In [83]:
df_final = pd.concat([df_final, df_to_append])

## save

In [84]:
len(df_final)

2103

In [85]:


df_final = df_final.drop_duplicates()

In [86]:
len(df_final)

2103

In [87]:
df_final = df_final.dropna(how="all", axis=1)

In [88]:
len(df_final)

2103

In [89]:
df_final.tail()

,extrStartDate,extrStartTime,extrEndTime,extrEndDate,extrMultipleDaysDetails,extrCity,extrStreetHouseNumber,extrLocationDetails,extrLocationName,isOnMultipleDays,origin_id,Event_category
486,2024-10-25,15,23,2024-10-31,"Montag bis Donnerstag 15:00 - 22:00 Uhr\nFreitag 15:00 - 23:00 Uhr\nSamstag 14:00 - 23:00 Uhr\nSonntag 14:00 - 22:00 Uhr\nSCHLIESSTAGE 21.10., 22.10., 28.10., 29.10.2024",Dresden,NaN,01067 Dresden Pieschener Allee 1,Volksfestgelände Pieschener Allee,True,45211,"Feste, Märkte & Messen"
486,2024-10-26,14,23,2024-10-31,"Montag bis Donnerstag 15:00 - 22:00 Uhr\nFreitag 15:00 - 23:00 Uhr\nSamstag 14:00 - 23:00 Uhr\nSonntag 14:00 - 22:00 Uhr\nSCHLIESSTAGE 21.10., 22.10., 28.10., 29.10.2024",Dresden,NaN,01067 Dresden Pieschener Allee 1,Volksfestgelände Pieschener Allee,True,45211,"Feste, Märkte & Messen"
486,2024-10-27,14,22,2024-10-31,"Montag bis Donnerstag 15:00 - 22:00 Uhr\nFreitag 15:00 - 23:00 Uhr\nSamstag 14:00 - 23:00 Uhr\nSonntag 14:00 - 22:00 Uhr\nSCHLIESSTAGE 21.10., 22.10., 28.10., 29.10.2024",Dresden,NaN,01067 Dresden Pieschener Allee 1,Volksfestgelände Pieschener Allee,True,45211,"Feste, Märkte & Messen"
486,2024-10-30,15,22,2024-10-31,"Montag bis Donnerstag 15:00 - 22:00 Uhr\nFreitag 15:00 - 23:00 Uhr\nSamstag 14:00 - 23:00 Uhr\nSonntag 14:00 - 22:00 Uhr\nSCHLIESSTAGE 21.10., 22.10., 28.10., 29.10.2024",Dresden,NaN,01067 Dresden Pieschener Allee 1,Volksfestgelände Pieschener Allee,True,45211,"Feste, Märkte & Messen"
486,2024-10-31,15,22,2024-10-31,"Montag bis Donnerstag 15:00 - 22:00 Uhr\nFreitag 15:00 - 23:00 Uhr\nSamstag 14:00 - 23:00 Uhr\nSonntag 14:00 - 22:00 Uhr\nSCHLIESSTAGE 21.10., 22.10., 28.10., 29.10.2024",Dresden,NaN,01067 Dresden Pieschener Allee 1,Volksfestgelände Pieschener Allee,True,45211,"Feste, Märkte & Messen"


In [90]:
df_final["Source"]="browse_ai"

In [91]:
# df.to_excel(f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.xlsx")

In [92]:
df_final.to_csv(f"{DATA_FOLDER}df_browse_ai final {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv")

In [93]:
df_final.to_pickle(f"{DATA_FOLDER}df_browse_ai final {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")